# AI Psychometrics (D'Urso and Haslbeck, 2024)

# The following script is a pilot script for analysing the correlation between personality scores obtained through open-source llm via zero-shot classification and real data.

## Step 1: Translating the english survey questions in the following languages:

- Chinese
- Italian

In [5]:
import pandas as pd 
import numpy as np

In [6]:
questions = ["I am the life of the party.",
              "I don't talk a lot.",
              "I feel comfortable around people.",
              "I keep in the background.",
              "I start conversations.",
              "I have little to say.",
              "I talk to a lot of different people at parties.",
              "I don't like to draw attention to myself.",
              "I don't mind being the center of attention.",
              "I am quiet around strangers.",
              "I get stressed out easily.",
              "I am relaxed most of the time.",
              "I worry about things.",
              "I seldom feel blue.",
              "I am easily disturbed.",
              "I get upset easily.",
              "I change my mood a lot.",
              "I have frequent mood swings.",
              "I get irritated easily.",
              "I often feel blue.",
              "I feel little concern for others.",
              "I am interested in people.",
              "I insult people.",
              "I sympathize with others' feelings.",
              "I am not interested in other people's problems.",
              "I have a soft heart.",
              "I am not really interested in others.",
              "I take time out for others.",
              "I feel others' emotions.",
              "I make people feel at ease.",
              "I am always prepared.",
              "I leave my belongings around.",
              "I pay attention to details.",
              "I make a mess of things.",
              "I get chores done right away.",
              "I often forget to put things back in their proper place.",
              "I like order.",
              "I shirk my duties.",
              "I follow a schedule.",
              "I am exacting in my work.",
              "I have a rich vocabulary.",
              "I have difficulty understanding abstract ideas.",
              "I have a vivid imagination.",
              "I am not interested in abstract ideas.",
              "I have excellent ideas.",
              "I do not have a good imagination.",
              "I am quick to understand things.",
              "I use difficult words.",
              "I spend time reflecting on things.",
              "I am full of ideas." ]

labels_questions = ["EXT1"	,
                    "EXT2"	,
                    "EXT3"	,
                    "EXT4"	,
                    "EXT5"	,
                    "EXT6"	,
                    "EXT7"	,
                    "EXT8"	,
                    "EXT9"	,
                    "EXT10",
                    "EST1"	,
                    "EST2"	,
                    "EST3"	,
                    "EST4"	,
                    "EST5"	,
                    "EST6"	,
                    "EST7"	,
                    "EST8"	,
                    "EST9"	,
                    "EST10",
                    "AGR1"	,
                    "AGR2"	,
                    "AGR3"	,
                    "AGR4"	,
                    "AGR5"	,
                    "AGR6"	,
                    "AGR7"	,
                    "AGR8"	,
                    "AGR9"	,
                    "AGR10",
                    "CSN1"	,
                    "CSN2"	,
                    "CSN3"	,
                    "CSN4"	,
                    "CSN5"	,
                    "CSN6"	,
                    "CSN7"	,
                    "CSN8"	,
                    "CSN9"	,
                    "CSN10",
                    "OPN1"	,
                    "OPN2"	,
                    "OPN3"	,
                    "OPN4"	,
                    "OPN5"	,
                    "OPN6"	,
                    "OPN7"	,
                    "OPN8"	,
                    "OPN9"	,
                    "OPN10"]

big5_qs = pd.DataFrame(list(zip(labels_questions, questions)), columns = ['item', 'question'])

In [7]:
# response options (for now we use default from https://github.com/Alheimsins/b5-johnson-120-ipip-neo-pi-r) but we may consider translating
languages = ['en', 'it', 'zh']

# Response options below. Copied from the .js files in the different language folders

candidate_labels_en = [
    "Very Inaccurate",
    "Moderately Inaccurate",
    "Neither Accurate Nor Inaccurate",
    "Moderately Accurate",
    "Very Accurate",
]
candidate_labels_it = [
    "Molto in disaccordo",
    " In disaccordo",
    "Nè in accordo nè in disaccordo",
    "In accordo",
    " Molto in accordo",
]
candidate_labels_ch = ["错误", "失准", "普通", "准确", "精确"]

labels = pd.DataFrame(
    list(
        zip(
            candidate_labels_en,
            candidate_labels_it,
            candidate_labels_ch
        )
    ),
    columns=languages,
)
labels

,en,it,zh
0,Very Inaccurate,Molto in disaccordo,错误
1,Moderately Inaccurate,In disaccordo,失准
2,Neither Accurate Nor Inaccurate,Nè in accordo nè in disaccordo,普通
3,Moderately Accurate,In accordo,准确
4,Very Accurate,Molto in accordo,精确


In [8]:
#call translation package and translate questions
import translators as ts

translators_services = ['google', 'bing', 'baidu'] #translation services 
languages = ['it', 'zh'] #languages to translate the questions to 

#for loop to translate across different translation services, languages and add as columns to the original data set
for serv in translators_services:
    for lang in languages:
        temp = []
        for item in big5_qs['question']:
            temp.append(ts.translate_text(item, from_language='en', to_language=lang, translator=serv)) #from language is necessary because some translators (e.g., alibaba) default to origin country setting "china".
        big5_qs = pd.concat([big5_qs, pd.DataFrame(list(zip(temp)),columns =[lang + '_' + serv])], axis = 1)

In [9]:
big5_qs.head()

,item,question,it_google,zh_google,it_bing,zh_bing,it_baidu,zh_baidu
0,EXT1,I am the life of the party.,Sono la vita della festa.,我是聚会的生活。,Io sono l'anima della festa.,我是党的生命。,Io sono la vita della festa.,我是党的生命。
1,EXT2,I don't talk a lot.,Non parlo molto.,我说话不多。,Non parlo molto.,我不怎么说话。,Non parlo molto.,我不怎么说话。
2,EXT3,I feel comfortable around people.,Mi sento a mio agio con le persone.,我周围很舒服。,Mi sento a mio agio con le persone.,我和人在一起感觉很舒服。,Mi sento a mio agio con la gente.,和人在一起我感觉很舒服。
3,EXT4,I keep in the background.,Tengo in background.,我留在后台。,Rimango sullo sfondo.,我保持在后台。,Mi tengo sullo sfondo.,我一直在幕后。
4,EXT5,I start conversations.,Inizio le conversazioni.,我开始对话。,Inizio le conversazioni.,我开始对话。,Inizio conversazioni.,我开始交谈。


## Step 2: Let llm respond to questions in different languages

In [10]:
from transformers import pipeline

In [11]:
responses = []
languages = ['en', 'it', 'zh', 'it', 'zh', 'it', 'zh']

for item in range(0,len(languages)):
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    answerlist = list(labels[languages[item]])
    questionlist = big5_qs.iloc[:,item+1]
    for sequence_to_classify_english in questionlist:
        answer = classifier(sequence_to_classify_english, answerlist)
        print(answer)
        responses.append(answer)

{'sequence': 'I am the life of the party.', 'labels': ['Moderately Accurate', 'Very Accurate', 'Moderately Inaccurate', 'Neither Accurate Nor Inaccurate', 'Very Inaccurate'], 'scores': [0.40380170941352844, 0.3170546293258667, 0.18389591574668884, 0.06463424116373062, 0.03061349131166935]}
{'sequence': "I don't talk a lot.", 'labels': ['Moderately Accurate', 'Moderately Inaccurate', 'Very Inaccurate', 'Very Accurate', 'Neither Accurate Nor Inaccurate'], 'scores': [0.6016448736190796, 0.29415029287338257, 0.04612812027335167, 0.03574517369270325, 0.022331498563289642]}
{'sequence': 'I feel comfortable around people.', 'labels': ['Moderately Accurate', 'Moderately Inaccurate', 'Very Accurate', 'Neither Accurate Nor Inaccurate', 'Very Inaccurate'], 'scores': [0.6052239537239075, 0.21031251549720764, 0.12311545014381409, 0.03462200239300728, 0.02672610990703106]}
{'sequence': 'I keep in the background.', 'labels': ['Moderately Accurate', 'Moderately Inaccurate', 'Very Accurate', 'Very Inac

In [12]:
# Change all language labels to english

answers_all_english = pd.DataFrame(columns=candidate_labels_en)
answers_all_english["language"] = []
answers_all_english["model"] = []
answers_all_english["item"] = []
answers_all_english["item_index"] = []

for lan in range(0, len(languages)):
    # extract responses for different languages
    answers_temp = pd.DataFrame(
        [
            dict(zip(x["labels"], x["scores"]))
            for x in responses[lan * 50 : lan * 50 + 50]
        ]
    )
    # insert language
    answers_temp.insert(len(answers_temp.columns), "language", big5_qs.columns[lan+1])
    # insert model name
    answers_temp["model"] = list(np.repeat('model', 50))
    # insert item text
    answers_temp.insert(len(answers_temp.columns), "item", big5_qs["question"])
    answers_temp.insert(
        len(answers_temp.columns), "item_index", list(range(1, len(questionlist) + 1))
    )
    # adjust column names
    answers_temp = answers_temp.rename(
        columns=dict(zip(labels[languages[lan]], labels["en"]))
    )
    # concatenate
    answers_all_english = pd.concat([answers_all_english, answers_temp])

answers_all_english

,Very Inaccurate,Moderately Inaccurate,Neither Accurate Nor Inaccurate,Moderately Accurate,Very Accurate,language,model,item,item_index
0,0.030613,0.183896,0.064634,0.403802,0.317055,question,model,I am the life of the party.,1.0
1,0.046128,0.294150,0.022331,0.601645,0.035745,question,model,I don't talk a lot.,2.0
2,0.026726,0.210313,0.034622,0.605224,0.123115,question,model,I feel comfortable around people.,3.0
3,0.040123,0.281780,0.037826,0.588398,0.051873,question,model,I keep in the background.,4.0
4,0.044577,0.253531,0.043915,0.538469,0.119508,question,model,I start conversations.,5.0
...,...,...,...,...,...,...,...,...,...
45,0.169719,0.162800,0.169013,0.229350,0.269119,zh_baidu,model,I do not have a good imagination.,46.0
46,0.168213,0.185645,0.139338,0.265003,0.241800,zh_baidu,model,I am quick to understand things.,47.0
47,0.214714,0.123331,0.193304,0.231452,0.237199,zh_baidu,model,I use difficult words.,48.0
48,0.163455,0.156151,0.167139,0.234459,0.278795,zh_baidu,model,I spend time reflecting on things.,49.0


### Prepare Data set for Scoring
Specifically, we want to have item as columns, and, for each of the response options, we want to calculate the argmax (i.e., the category with the highest probability

In [13]:
df = answers_all_english

# Selecting the scoring columns
score_columns = [
    "Very Inaccurate",
    "Moderately Inaccurate",
    "Neither Accurate Nor Inaccurate",
    "Moderately Accurate",
    "Very Accurate",
]

# Finding the column with the highest score for each row
df["max_score"] = df[score_columns].idxmax(axis=1)

# Mapping column names to numeric scores
column_to_score = {
    "Very Inaccurate": 1,
    "Moderately Inaccurate": 2,
    "Neither Accurate Nor Inaccurate": 3,
    "Moderately Accurate": 4,
    "Very Accurate": 5,
}

df["score"] = df["max_score"].map(column_to_score)

# Pivoting the DataFrame
transformed_df = df.pivot_table(
    index=["language", "model"], columns="item", values="score", aggfunc="first"
)

# Resetting the index
transformed_df.reset_index(inplace=True)

# Display the transformed DataFrame
transformed_df

item,language,model,I am always prepared.,I am easily disturbed.,I am exacting in my work.,I am full of ideas.,I am interested in people.,I am not interested in abstract ideas.,I am not interested in other people's problems.,I am not really interested in others.,...,I pay attention to details.,I seldom feel blue.,I shirk my duties.,I spend time reflecting on things.,I start conversations.,I sympathize with others' feelings.,I take time out for others.,I talk to a lot of different people at parties.,I use difficult words.,I worry about things.
0,it_baidu,model,4,2,4,4,4,2,2,2,...,4,2,4,2,4,4,4,1,2,2
1,it_bing,model,4,2,4,4,4,2,2,2,...,4,2,4,2,4,4,4,4,2,2
2,it_google,model,4,2,4,4,4,2,2,2,...,4,2,4,2,4,4,4,1,2,2
3,question,model,4,2,5,4,4,4,4,4,...,4,4,2,4,4,4,4,4,2,4
4,zh_baidu,model,2,5,5,5,5,4,5,1,...,4,5,3,5,5,5,5,5,5,4
5,zh_bing,model,2,5,5,5,5,4,5,5,...,4,5,5,5,1,5,5,5,4,5
6,zh_google,model,2,5,5,5,5,4,5,5,...,4,5,3,5,1,5,5,5,1,5


In [14]:
transformed_df.columns

Index(['language', 'model', 'I am always prepared.', 'I am easily disturbed.',
       'I am exacting in my work.', 'I am full of ideas.',
       'I am interested in people.', 'I am not interested in abstract ideas.',
       'I am not interested in other people's problems.',
       'I am not really interested in others.',
       'I am quick to understand things.', 'I am quiet around strangers.',
       'I am relaxed most of the time.', 'I am the life of the party.',
       'I change my mood a lot.', 'I do not have a good imagination.',
       'I don't like to draw attention to myself.',
       'I don't mind being the center of attention.', 'I don't talk a lot.',
       'I feel comfortable around people.',
       'I feel little concern for others.', 'I feel others' emotions.',
       'I follow a schedule.', 'I get chores done right away.',
       'I get irritated easily.', 'I get stressed out easily.',
       'I get upset easily.', 'I have a rich vocabulary.',
       'I have a soft heart

## Step 3: Collect empirical data for each of the languages that were used by the llm model

In [18]:
kag_b5 = pd.read_csv('data-final.csv', sep="\t")

In [42]:
import copy 
#collect different data sets for each of the countries
kag_b5_us = kag_b5[kag_b5['country'] == 'US']
kag_b5_it = kag_b5[kag_b5['country'] == 'IT']
kag_b5_ch = kag_b5[kag_b5['country'] == 'CN']

In [67]:
kag_b5_it = kag_b5_it[:50].rename(columns = dict(zip(big5_qs['item'], big5_qs['question'])))
kag_b5_it.iloc[:,:50].describe()

,I am the life of the party.,I don't talk a lot.,I feel comfortable around people.,I keep in the background.,I start conversations.,I have little to say.,I talk to a lot of different people at parties.,I don't like to draw attention to myself.,I don't mind being the center of attention.,I am quiet around strangers.,...,I have a rich vocabulary.,I have difficulty understanding abstract ideas.,I have a vivid imagination.,I am not interested in abstract ideas.,I have excellent ideas.,I do not have a good imagination.,I am quick to understand things.,I use difficult words.,I spend time reflecting on things.,I am full of ideas.
count,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,2.420000,2.84000,2.960000,3.240000,2.900000,2.520000,2.460000,3.46000,2.800000,3.720000,...,3.900000,1.680000,4.080000,1.88000,3.680000,1.720000,3.760000,3.300000,4.340000,3.940000
std,1.213697,1.44787,1.384462,1.363669,1.403349,1.265556,1.487499,1.35842,1.355262,1.229568,...,1.147313,0.793854,1.175255,1.06215,1.077412,1.050559,1.270706,1.111168,0.981669,1.114103
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.00000,2.000000,2.000000,2.000000,2.000000,1.000000,3.00000,2.000000,3.000000,...,3.000000,1.000000,4.000000,1.00000,3.000000,1.000000,3.000000,2.250000,4.000000,3.000000
50%,3.000000,3.00000,3.000000,3.000000,3.000000,2.000000,2.000000,4.00000,3.000000,4.000000,...,4.000000,2.000000,4.000000,2.00000,4.000000,1.000000,4.000000,4.000000,5.000000,4.000000
75%,3.000000,4.00000,4.000000,4.000000,4.000000,3.000000,4.000000,5.00000,4.000000,5.000000,...,5.000000,2.000000,5.000000,2.75000,4.000000,2.000000,5.000000,4.000000,5.000000,5.000000
max,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,...,5.000000,4.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [66]:
transformed_df[list(kag_b5_us.columns[:50])]

item,I am the life of the party.,I don't talk a lot.,I feel comfortable around people.,I keep in the background.,I start conversations.,I have little to say.,I talk to a lot of different people at parties.,I don't like to draw attention to myself.,I don't mind being the center of attention.,I am quiet around strangers.,...,I have a rich vocabulary.,I have difficulty understanding abstract ideas.,I have a vivid imagination.,I am not interested in abstract ideas.,I have excellent ideas.,I do not have a good imagination.,I am quick to understand things.,I use difficult words.,I spend time reflecting on things.,I am full of ideas.
0,4,2,4,4,4,2,1,2,2,2,...,5,2,4,2,4,2,4,2,2,4
1,4,2,4,2,4,2,4,2,2,5,...,5,2,4,2,4,2,4,2,2,4
2,4,2,4,2,4,2,1,2,2,5,...,5,2,4,2,4,2,2,2,2,4
3,4,4,4,4,4,2,4,4,4,4,...,4,2,2,4,5,2,4,2,4,4
4,5,1,5,5,5,5,5,5,5,1,...,4,4,5,4,5,5,4,5,5,5
5,5,1,4,5,1,5,5,5,5,1,...,4,4,5,4,5,5,5,4,5,5
6,5,1,1,1,1,1,5,4,5,1,...,4,4,5,4,5,5,4,1,5,5
